# Task 1 Reconstruct the Original Meeting Transcripts

#### Student Name: Xiaofan Zhang
#### Student ID: 28338707

Date: 03/06/2018

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,: 
* re 
* os
* xml.etree.ElementTree 

Running Times:
It needs at least 20 mins to run all files

### Import Library

In [ ]:
import xml.etree.ElementTree 
import re
import os

In [ ]:
xml_file_path = "./topics"

## Step 1: Function for creating dict{topic: corresponding word file name}

In [ ]:
# function for create dict{topic: corresponding word file name}
def getDic(x):
    topic = ""
    allList = []
    
    topic_file = xml.etree.ElementTree.parse(x)
    topic_root = topic_file.getroot()
    for child in topic_root:
        allIn = {}
        # topic name 
        topic = child.attrib[list(child.attrib.keys())[0]]
        word_id = []
        # get all child include sub-topic
        for c in child.iter():
            if list(c.attrib.keys())[0] == 'href':
                ca = c.attrib['href']
                pattern_word = re.compile(r'(\w{2}\d{4}\w{1}.\w{1}.words|\w{2}\d{4}.\w{1}.words).xml')
                word_id += (pattern_word.findall(ca))
            # topic name as key and word file name as value
            allIn= {topic : word_id}
        allList.append(allIn)
    return(allList)

# save all dictionary in a list
xml_file_path = "./topics"
listDix = []
for xfile in os.listdir(xml_file_path): 
    listDix.append(getDic("topics/"+ xfile))
    
lists = listDix

## Step 2: Function for creating dict{topic: corresponding word file text boundaries}

In [ ]:
# function for create dict{topic: corresponding word file text boundaries}
def getDicste(x):
    topic = ""
    allList = []
    
    topic_file = xml.etree.ElementTree.parse(x)
    topic_root = topic_file.getroot()
    
    for child in topic_root:
        allIn = {}
        topic = child.attrib[list(child.attrib.keys())[0]]
        word_ste1 = []
        # get all child include sub-topic
        for c in child.iter():
            if list(c.attrib.keys())[0] == 'href':
                ca = c.attrib['href']
                word_num = re.compile(r'words(\d{1,})')
                word_ste1.append(word_num.findall(ca))
        
        # topic name as key and word file boundaries as value
        allIn= {topic : word_ste1}
        allList.append(allIn)
    return(allList)
# save all dictionary in a list
listDicste = []
for xfile in os.listdir(xml_file_path): 
    listDicste.append(getDicste("topics/"+ xfile))
    
lists1 = listDicste

## Step 3: Function for generating corresponding segment boundaries


In [ ]:
# function for generating dic{segment : corresponding segment boundaries}
# seg_file_path = "./segments"
def getSefNum(x):
    file = xml.etree.ElementTree.parse(x)
    root = file.getroot()
    seg = []
    word_name1 = []
    for segment in root:
        for child in segment.findall("{http://nite.sourceforge.net/}child"):
            child_attr = child.attrib['href']
            seg_pos = re.compile(r'(\w{2}\d{4}\w.\w.words\d{1,}|\w{2}\d{4}.\w{1}.words\d{1,})')
            seg.append(seg_pos.findall(child_attr))
            
    return (seg)


## Step 4: Combine two topic dictionaries with segment and generate files

In [ ]:
for atopic in range(len(lists)):
    # topic file name for generate file
    file_obj = re.findall(r"(\w{2}\d{4}\w{1}|\w{2}\d{4}).topic",list(lists[atopic][0].keys())[0])[0]

    _str = []
    for l in range(len(lists[atopic])):
        values = list(lists[atopic][l].values())[0] # list of str 'ES2002a.B.words'
        num = list(lists1[atopic][l].values())[0]# list of lists
        word_str = ""
        for v in range(len(values)):
            seg_list = []
            wfile = xml.etree.ElementTree.parse("words/" + values[v] + ".xml").getroot()
            
            seg_list1 = ""
            seg_list1 = getSefNum("segments/" + re.findall(r"(\w{2}\d{4}\w{1}.\w{1}|\w{2}\d{4}.\w{1})", values[v])[0] + ".segments.xml")# seg_list1
            for k in seg_list1:
                seg_list.append(k[-1])# only get the last number of segments for add "\n"
            for word in wfile.getchildren():
                if len(num[v]) == 2:# two words range e.g.[0,71]
                    if word.attrib['{http://nite.sourceforge.net/}id'] in list(values[v] + str(i) for i in range(int(num[v][0]), int(num[v][1])+1)):
                        if word.attrib['{http://nite.sourceforge.net/}id'] in seg_list: 
                            if word.text is None:# ignore vocalsound but segment
                                word_str += "\n"
                            else:
                                word_str += " " + word.text+ "\n"
                        else:
                            if word.text is None:
                                pass
                            else:                
                                word_str += " " + word.text
                else: # one words range e.g.[72]
                    if word.attrib['{http://nite.sourceforge.net/}id'] in list(values[v] + str(i) for i in range(int(num[v][0]), int(num[v][0])+1)):
                        if word.attrib['{http://nite.sourceforge.net/}id'] in seg_list: 
                            if word.text is None:
                                word_str += "\n"
                            else:
                                word_str += " " + word.text+ "\n"
                        else:
                            if word.text is None:
                                pass
                            else: 
                                word_str += " " + word.text
            
            word_str = word_str+"\n"
                
        word_str = word_str+"\n"
        # delete any empty lines
        word_str = word_str.replace('\n\n\n\n\n\n', '\n')                    
        word_str = word_str.replace('\n\n\n\n\n', '\n')                    
        word_str = word_str.replace('\n\n\n\n', '\n')                    
        word_str = word_str.replace('\n\n\n', '\n')
        word_str = word_str.replace('\n\n', '\n')
        if word_str[0] == "\n":
            word_str = word_str[1:]

       
        _str.append(word_str)
        a = ""
        # add topical boundries
        a= "**********\n".join(_str) + "**********"

    
    file = open("txt_files3/"+file_obj+".txt","w")
    file.write(a)
    file.close()

   